In [ ]:
'''
타이타닉 탑승자 데이터
- PassengerId 
    - 탑승자 데이터 일련 번호
- Survived
    - 생존 여부 
        0 : 사망
        1 : 생존
- Pclass
    - 티켓의 선실 등급
        - 1 : 일등석
        - 2 : 이등석
        - 3 : 삼등석
- Name
    - 탑승자 이름
- Sex
    - 탑승자 성별
- Age
    - 탑승자 나이
- SibSp
    - 같이 탑승한 형제자매 또는 배우자 인원 수
- Parch
    - 같이 탑승한 부모 또는 어린이 인원 수
- Ticket
    - 티켓 번호
- Fare
    - 요금
- Cabin
    - 선실 번호
- Embarked
    - 중간 정착 항구
        - C : Cherbourg
        - Q : Queenstown
        - S : Southampton
'''

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

titanic_df = pd.read_csv("D:/대학원/머신러닝 특론/타이타닉 생존자 데이터셋/train.csv")
titanic_df.head(3)

In [ ]:
print(titanic_df.info())
'''
891 non-null -> 891개의 데이터가 Null이 아니다.
Age
- 714 non-null -> 714개의 데이터가 Null이 아니다. 즉, Null인 데이터가 존재한다.
'''

In [ ]:
'''
Age는 평균 나이, Cabin과 Embardked는 N으로 Null을 변경한다.
'''
titanic_df["Age"].fillna(titanic_df["Age"].mean(), inplace=True)
titanic_df["Cabin"].fillna("N", inplace=True)
titanic_df["Embarked"].fillna("N", inplace=True)
print(titanic_df.isnull().sum()) # Null이 없어진 것을 확인할 수 있다.

In [ ]:
'''
Sex, Cabin, Embarked의 값 분포 확인
'''
print(titanic_df["Sex"].value_counts())
print(titanic_df["Cabin"].value_counts())
print(titanic_df["Embarked"].value_counts())

In [ ]:
'''
Cabin의 앞글자만 가져와서 구분한다.
'''
titanic_df["Cabin"] = titanic_df["Cabin"].str[:1] # Cabin의 첫글자(앞글자)만 가져온다.
print(titanic_df["Cabin"].value_counts())

In [ ]:
titanic_df.groupby(["Sex", "Survived"])["Survived"].count()
'''
여성 생존율 = 233 / (233 + 81 = 314) => 74.2%
남성 생존율 = 109 / (468 + 109 = 577) => 18.9%
'''

In [ ]:
import seaborn as sns
# 성별에 따른 생존율 시각화
sns.barplot(x="Sex", y="Survived", data=titanic_df)

In [ ]:
# 객실 등급과 성별에 따른 생존율
sns.barplot(x="Pclass", y="Survived", hue="Sex", data=titanic_df)

'''
hue
- hue 매개변수는 데이터를 여러 그룹으로 나누어 각 그룹에 대한 정보를 다른 색상에 바로 표시할 때 사용한다.
- 위의 코드를 예로 들면, Pclass에 대하여 남성과 여성의 생존율을 표시한다.
- 즉, hue는 데이터의 카테고리를 구분하는 데 사용되며, 이를 통해 같은 x 값에 대한 여러 y값들을 각각 다른 색상으로 표시할 수 있다.
'''

In [ ]:
# 나이에 따른 생존율
def get_category(age):
    cat = ""
    if age <= -1: cat="Unknown"
    elif age <= 5: cat = "Baby"
    elif age <= 12: cat = "Child"
    elif age <= 18: cat = "Teenager"
    elif age <= 25: cat = "Student"
    elif age <= 35: cat = "Young Adult"
    elif age <= 60: cat = "Adult"
    else: cat = "Elderly"
    return cat

plt.figure(figsize=(10, 6))
groups_name = ["Unknown", "Baby", "Child", "Teenager", "Student", "Young Adult", "Adult", "Elderly"]

titanic_df["Age_cat"] = titanic_df["Age"].apply(lambda x:get_category(x))
sns.barplot(x="Age_cat", y="Survived", hue="Sex", data=titanic_df, order=groups_name)
titanic_df.drop("Age_cat", axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_feature(df):
    features = ["Cabin", "Sex", "Embarked"]
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

titanic_df = encode_feature(titanic_df)
titanic_df.head()
'''
LabelEncoder
- sklearn.preprocessing 모듈에 있는 클래스이다.
- 카테고리형 레이블을 정수형으로 변환하는데 사용된다.

features 리스트에 인코딩할 특성들의 이름을 지정한다.

for문을 돌려 LabelEncoder 객체 le를 생성하고, fit 메서드를 사용하여 특성의 모든 유니크한 값을 학습한다.
그 후, transform 메서드를 사용하여 각 레이블을 정수로 변환한다. 변환된 값은 원래의 데이터프레임에 다시 저장된다.

encode_feature()를 호출하여 titanic_df 데이터프레임의 특성들을 인코딩한다.
- Cabin, Sex, Embarked가 정수형으로 변경된 것을 확인할 수 있다.
'''

In [ ]:
# 함수 구현
def fillna(df):
    df["Age"].fillna(df["Age"].mean(), inplace = True)
    df["Cabin"].fillna("N", inplace = True)
    df["Embarked"].fillna("N", inplace = True)
    df["Fare"].fillna(0, inplace=True)
    return df

def drop_features(df):
    df.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace = True)
    return df

def format_features(df):
    df["Cabin"] = df["Cabin"].str[:1]
    features = ["Cabin", "Sex", "Embarked"]
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

titanic_df = pd.read_csv("D:/대학원/머신러닝 특론/타이타닉 생존자 데이터셋/train.csv")
y_titanic_df = titanic_df["Survived"]
X_titanic_df = titanic_df.drop("Survived", axis=1)
X_titanic_df = transform_features(X_titanic_df)

In [ ]:
# 성능 평가
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2)

dt_clf = DecisionTreeClassifier()
rf_clf = RandomForestClassifier()
lr_clf = LogisticRegression()

dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print(accuracy_score(y_test, dt_pred))

rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print(accuracy_score(y_test, rf_pred))

lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print(accuracy_score(y_test, lr_pred))

In [ ]:
# 교차 검증
from sklearn.model_selection import KFold

def exec_kfold(clf, folds=5):
    kfold = KFold(n_splits=folds)
    scores = []

    for iter_count, (train_index, test_index) in enumerate(kfold.split(X_titanic_df)):
        X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
        y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print(iter_count, accuracy)
    
    print(np.mean(scores))

exec_kfold(dt_clf, folds=5)
'''
enumerate() 함수는 반복 가능한 객체(리스트, 튜플 문자열)을 인자로 받아, 인덱스와 해당 인덱스의 값을 반복하는 반복자(iterator)를 생성한다.
이 함수는 주로 반복문에서 사용되며, 반복문을 통해 객체의 각 요소에 접근할 때 해당 요소의 인덱스도 함께 얻을 수 있게 한다.
- 즉, 반복문에서 현재의 인덱스와 값을 동시에 얻을 수 있다.
'''


In [ ]:
'''
cross_val_score
- 위의 교차 검증 과정을 하나의 함수만들어 사용한다.
'''
from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, X_titanic_df, y_titanic_df, cv=5)
for iter_count, accuracy in enumerate(scores):
    print(iter_count, accuracy)
print(np.mean(scores))

In [ ]:
# 최적의 하이퍼 파라미터
from sklearn.model_selection import GridSearchCV

parameters = {
    "max_depth" : [2, 3, 5, 10],
    "min_samples_split" : [2, 3, 5],
    "min_samples_leaf" : [1, 5, 8]
}
grid_dclf = GridSearchCV(dt_clf, param_grid = parameters, scoring="accuracy", cv=5)
grid_dclf.fit(X_train, y_train)

print(grid_dclf.best_params_)
print(grid_dclf.best_score_)
best_dclf = grid_dclf.best_estimator_

dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpredictions)
print(accuracy)

'''
parameters = {
    "max_depth" : [2, 3, 5, 10],
    "min_samples_split" : [2, 3, 5],
    "min_samples_leaf" : [1, 5, 8]
}
- GridSearchCV에서 사용할 하이퍼파라미터의 값들을 딕셔너리 형태로 정의한다.

grid_dclf = GridSearchCV(dt_clf, param_grid = parameters, scoring="accuracy", cv=5)
- GridSearchCV 객체를 생성한다.
    - dt_clf는 탐색할 결정 트리 모델이다.
    - param_grid는 탐색할 하이퍼파라미터의 그리드이다.
    - scoring은 모델 평가 기준이다.
    - cv는 교차 검증 폴드 수이다.

grid_dclf.fit(X_train, y_train)
- GridSerachCV를 사용하여 모델을 학습시킨다.
- 이 과정에서 모든 하이퍼파라미터 조합에 대해 교차 검증이 수행되며, 최적의 하이퍼파라미터가 선택된다.

print(grid_dclf.best_params_)
print(grid_dclf.best_score_)
- 최적의 하이퍼파라미터와 해당 하이퍼파라미터에서의 평균 교차 검증 점수를 출력한다.

best_dclf = grid_dclf.best_estimator_
- 최적의 하이퍼파라미터로 학습된 모델을 추출한다.

dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpredictions)
print(accuracy)
- 최적의 모델을 사용하여 테스트 데이터셋에 대한 예측을 수행하고, 정확도를 계산하여 출력한다.

이런 과정을 거쳐, GridSearchCV를 통해 결정 트리 모델의 최적의 하이퍼파라미터를 찾고, 이를 사용하여 모델을 학습하고 평가하는 전체 과정을 수행할 수 있다.
'''

In [ ]:
'''
평가

정확도(Accuracy)
- 실제 데이터에서 예측 데이터가 얼마나 같은지 판단하는 지표이다.
    - 정확도 = 예측 결과가 실제와 동일한 데이터 건수 / 전체 예측 데이터 건수
- 이진 분류의 경우 데이터 구성에 따라 머신 러닝 모델의 성능을 왜곡할 수 있다.
'''

# 왜곡의 예
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator

def fillna(df):
    df["Age"].fillna(df["Age"].mean(), inplace = True)
    df["Cabin"].fillna("N", inplace = True)
    df["Embarked"].fillna("N", inplace = True)
    df["Fare"].fillna(0, inplace=True)
    return df

def drop_features(df):
    df.drop(["PassengerId", "Name", "Ticket"], axis=1, inplace = True)
    return df

def format_features(df):
    df["Cabin"] = df["Cabin"].str[:1]
    features = ["Cabin", "Sex", "Embarked"]
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

class MyDummyClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass

    def predict(self, X):
        pred = np.zeros((X.shape[0], 1))
        for i in range(X.shape[0]):
            if X["Sex"].iloc[i] == 1: pred[i] = 0
            else: pred[i] = 1
        return pred
    '''
    클래스의 인스턴스 메서드에서
    self를 명시적으로 넣어주지 않으면, 메서드 내에서 현재 인스턴스에 접근할 수 없게 된다.
    즉, 인스턴스 변수를 사용하거나 다른 인스턴스에서 메서드를 호출할 수 없게 된다.
    '''

titanic_df = pd.read_csv("D:/대학원/머신러닝 특론/타이타닉 생존자 데이터셋/train.csv")
y_titanic_df = titanic_df["Survived"]
X_titanic_df = titanic_df.drop("Survived", axis=1)
X_titanic_df = transform_features(X_titanic_df)

myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print(accuracy_score(y_test, mypredictions))

In [ ]:
'''
앞의 예시에서 단순히 Sex가 1이면 죽음(0), 아니면 생존(1)로 처리하여도
정확도가 0.7 ~ .8의 높은 값을 얻는다.

레이블 값 분포가 불균형한 경우 적합한 평가가 아니다.
- 100개의 데이터 중 90개는 레이블이 0, 10개는 1인 경우
    - 예측 결과를 무조건 0으로 반환해도 정확도가 90%가 나온다.

왜곡의 예
- MNIST 데이터 중 레이블 7만 True, 나머지는 False
- 이진 분류 문제로 변경
- 전체 데이터의 10%는 True, 90%는 False
    - 예측 값을 무조건 0으로 반환해도 정확도가 90%이다.
'''
from sklearn.datasets import load_digits

class MyFakeClassifier(BaseEstimator):
    def fit(self, X, y):
        pass
    def predict(self ,X):
        return np.zeros((X.shape[0], 1), dtype=int)

digits = load_digits()
y = (digits.target == 7).astype(int)

X_train, X_test, y_train, y_test = train_test_split(digits.data, y)
print(y_test.shape)
print(pd.Series(y_test).value_counts())

clf = MyFakeClassifier()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
print(accuracy_score(y_test, predictions))

In [ ]:
'''
오차 행렬(Confusion Matrix)
- 예측 오류가 얼마인지와 어떤 유형의 예측 오류가 발생하는지 나타낸다.
- TN(True Negative)
    - Negative로 예측했는데 그게 맞는 경우
- FP(False Positive)
    - Positive로 예측했는데 그게 틀린 경우
- FN(False Negative)
    - Negative로 예측했는데 그게 틀린 경우
    - 즉, 원래 데이터는 참인경우
- TP(True Positive)
    - Positive로 예측했는데 그게 맞는 경우
'''
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions))

'''
정확도 = 예측 결과가 실제와 동일한 데이터 건수 / 전체 예측 데이터 건수
=  TN + TP / TN + FP + FN + TP



재현율 = TP / FN + TP
'''

In [ ]:
'''
정밀도(Precision)
- TP / FP + TP
- Positive로 예측한 것 중 맞게 예측한 것의 비율
- 양성 예측도(Positive 예측 성능을 더욱 정밀하게 측정)라고도 한다.
- 실제로는 Negative인데 Positive로 잘못 판단하면 큰일 나는 경우에 적용한다.
- 예) 스팸 메일 판단 모델
    - Positive를 Negative로 판단하면 불편함 정도
    - Negative를 Positive로 판단하면 업무 차질
'''

In [ ]:
'''
재현율(Recall)
- TP / FN + TP
- 정답이 Positive인 것 중 맞게 예측한 것의 비율
- 민감도(Sensitivity), TPR(True Positive Rate)라고도 한다.
- 실제로는 Positive인데 Negative로 잘못 판단하면 큰일 나는 경우에 적용한다.
- 예) 암 판단 모델
    - Positive를 Negative로 판단하면 생명에 지장
    - Negative를 Positive로 판단하면 재검사 시도
- 예) 금융 사기 적발 모델
    - Positive를 Negative로 판단하면 회사에 손해
    - Negative를 Positive로 판단하면 재검사 시도
'''

In [ ]:
'''
정밀도 vs 재현율
- 정밀도 = TP / FP + TP
- 재현율 = TP / FN + TP

- 분자는 같다.
- 정밀도는 FP를 낮추는데 초점을 둔다.
- 재현율은 FN을 낮추는데 초점을 둔다.

- 정밀도와 재현율 모두 높은 것이 좋다.
- 둘 중 하나만 높고 다른 하나는 낮으면 안좋다.
'''

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv("D:/대학원/머신러닝 특론/타이타닉 생존자 데이터셋/train.csv")

df.drop(["PassengerId", "Name", "Ticket"], axis = 1, inplace = True)
df["Age"].fillna(df["Age"].mean(), inplace = True)
df["Cabin"].fillna("N", inplace=True)
df["Embarked"].fillna("N", inplace = True)
df["Cabin"] = df["Cabin"].str[:1]

encoder = LabelEncoder()
df["Cabin"] = encoder.fit_transform(df["Cabin"])
df["Sex"] = encoder.fit_transform(df["Sex"])
df["Embarked"] = encoder.fit_transform(df["Embarked"])

y = df["Survived"]
X = df.drop("Survived", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver = "liblinear")
lr.fit(X_train, y_train)
y_hat = lr.predict(X_test)

print(confusion_matrix(y_test, y_hat))
print(accuracy_score(y_test, y_hat))
print(precision_score(y_test, y_hat))
print(recall_score(y_test, y_hat))

In [ ]:
'''
트레이드오프(Trade-off)
- 정밀도 / 재현율이 강조되어야 하는 경우
    - 분류의 임곗값을 조정하면 바뀔 수 있다.
- 한쪽을 높이면 다른 쪽이 떨어질 수 있다.
- predict_proba()
    - 테스트 데이터에 대해 예측 확률 반환
'''
proba = lr.predict_proba(X_test)
result = np.concatenate([proba, y_hat.reshape(-1, 1)], axis=1)
print(result)
# 0일 확률과 1일 확률 중 더 높은 것을 선택한다.

In [ ]:
'''
Binarizer 클래스
- fit_transform():
    - 지정된 문턱 값보다 크면 1, 아니면 0으로 변환한다.
'''
from sklearn.preprocessing import Binarizer
tmp = [[1, -1, 2],
       [2, 0, 0],
       [0, 1.1, 1.2]]
binarizer = Binarizer(threshold=1.1) # tmp값이 threshold값 보다 크면 1 아니면 0으로 반환한다.
print(binarizer.fit_transform(tmp))
'''
[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]
'''

In [ ]:
pr = proba[:, 1].reshape(-1, 1) # pr = proba에서 모든 행을 가져오고, 1번째 열만 가져온다.

y_hat = Binarizer(threshold=0.5).fit_transform(pr) # pr이 0.5보다 크면 1 아니면 0으로 반환한다.
print(confusion_matrix(y_test, y_hat))
print(accuracy_score(y_test, y_hat))
print(precision_score(y_test, y_hat))
print(recall_score(y_test, y_hat))


In [ ]:
for th in [0.4, 0.45, 0.5, 0.55, 0.6]:
    y_hat = Binarizer(threshold=th).fit_transform(pr)
    print(confusion_matrix(y_test, y_hat))
    print(accuracy_score(y_test, y_hat))
    print(precision_score(y_test, y_hat))
    print(recall_score(y_test, y_hat))

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, ths = precision_recall_curve(y_test, pr)

N = ths.shape[0]
index = np.arange(0, N, 15)
print(index)

print(ths[index])
print(precisions[index])
print(recalls[index])

In [ ]:
import matplotlib.pyplot as plt

N = ths.shape[0]
plt.plot(ths, precisions[:N], "--")
plt.plot(ths, recalls[:N])
plt.grid()

In [ ]:
'''
정밀도(Precision)와 재현율(Recall)의 맹점
- 환자 1000명 중 확실한 Positive 징후 환자 1명만 Positive, 나머지는 Negative로 예측한다.
    - TP = 1, FP = 0 => 정밀도 100%

- 모든 환자를 Positive로 예측, 실제 양성은 30명
    - FN = 0, TP = 30 => 재현율 100%
'''

In [ ]:
'''
F1 스코어
- 정밀도와 재현율 결합
- F1 = 2 / ((1 / recall) = (1 / precision)) = 2 * (precision * recall) / (precision + recall)
    - 정밀도 0.9, 재현율 0.1 => F1 = 0.18
    - 정밀도 0.5, 재현율 0.5 => F1 = 0.5
'''
from sklearn.metrics import f1_score

for th in [0.4, 0.45, 0.5, 0.55, 0.6]:
    y_hat = Binarizer(threshold=th).fit_transform(pr)
    print(confusion_matrix(y_test, y_hat))
    print(accuracy_score(y_test, y_hat))
    print(precision_score(y_test, y_hat))
    print(recall_score(y_test, y_hat))
    print(f1_score(y_test, y_hat))

In [ ]:
'''
ROC 곡선(Reciver Operation Characteristic Curve)
- FPR(False Positive Rate)이 변할 때 TPR(True Positive Rate)이 어떻게 변하는지 나타내는 곡선이다.
- X축 : FPR
    - False Positive / Total Number of Actual Negative Events
    - 실제로 N인데 P로 분류한 비율
- Y축 : TPR
    - True Positive / Total Number of Actual Positive Events
    - 실제로 P인데 P로 분류한 비율
'''
from sklearn.metrics import roc_curve

fprs, tprs, ths = roc_curve(y_test, pr)
plt.plot(fprs, tprs)
plt.plot([0, 1], [0, 1], 'k--')
# 가운데 점선에 가까울수록 성능이 떨어진다.
'''
AUC(Area Under Curve)
- ROC 곡선의 아래의 면적이다.
    - 1에 가까울 수록 좋다.
'''

In [ ]:
from sklearn.metrics import roc_auc_score

for th in [0.4, 0.45, 0.5, 0.55, 0.6]:
    y_hat = Binarizer(threshold=th).fit_transform(pr)
    print(confusion_matrix(y_test, y_hat))
    print(accuracy_score(y_test, y_hat))
    print(precision_score(y_test, y_hat))
    print(recall_score(y_test, y_hat))
    print(f1_score(y_test, y_hat))
    print(roc_auc_score(y_test, pr))

In [ ]:
from sklearn.metrics import roc_auc_score

print(confusion_matrix(y_test, y_hat))
print(accuracy_score(y_test, y_hat))
print(precision_score(y_test, y_hat))
print(recall_score(y_test, y_hat))
print(f1_score(y_test, y_hat))
print(roc_auc_score(y_test, pr))

In [ ]:
'''
피마 인디언 당뇨병 분류
- Pregnancies
    - 임신 횟수
- Glucose
    - 포도당 부하 검사 수치
- BloodPressure
    - 혈압
- SkinThickness
    - 팔 삼두근 뒤쪽의 피하지방 측정값
- Insulin
    - 혈청 인슐린
- BMI
    - 체질량지수
- DiabetesPedigreeFunction
    - 당뇨 내력 가중치 값
- Age
    - 나이
- Outcome
    - 클래스 결정 값(0 또는 1)
'''

In [ ]:
diabetes = pd.read_csv("D:/대학원/머신러닝 특론/피마 인디언 당뇨병 분류/diabetes.csv")
print(diabetes["Outcome"].value_counts())
# 당뇨병 500명, 당뇨 x 268명

In [ ]:
diabetes.head()

In [ ]:
diabetes.info()

In [ ]:
X = diabetes.iloc[:, :-1] # Outcome 열 제외
y = diabetes.iloc[:, -1] # Outcome 열만 사용

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

lr = LogisticRegression(solver = "liblinear")
lr.fit(X_train, y_train)
y_hat = lr.predict(X_test)
y_prob = lr.predict_proba(X_test)[:, 1]

print(confusion_matrix(y_test, y_hat))
print(accuracy_score(y_test, y_hat))
print(precision_score(y_test, y_hat))
print(recall_score(y_test, y_hat))
print(f1_score(y_test, y_hat))
print(roc_auc_score(y_test, y_prob))

In [ ]:
precisions, recalls, ths = precision_recall_curve(y_test, y_prob)
N = ths.shape[0]
plt.plot(ths, precisions[:N], "--")
plt.plot(ths, recalls[:N])
plt.grid()


In [ ]:
diabetes.describe() # min 값이 0인 피처가 많다. => 이상치

In [ ]:
plt.hist(diabetes["Glucose"], bins=10)

In [ ]:
# min 값이 0인 피처 추출
# - 0의 건수
# 전체 데이터 건수 대비 비율
zero_features = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
total_count = diabetes["Glucose"].count()

for feature in zero_features:
    zero_count = diabetes[diabetes[feature] == 0][feature].count()
    print(feature, zero_count, 100 * zero_count / total_count)

In [ ]:
# 0값을 평균 값으로 대체
from sklearn.preprocessing import StandardScaler

mean_zero_features = diabetes[zero_features].mean()
diabetes[zero_features] = diabetes[zero_features].replace(0, mean_zero_features)

X = diabetes.iloc[:, :-1]
y = diabetes.iloc[:, -1]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

lr = LogisticRegression()
lr.fit(X_train, y_train)
y_hat = lr.predict(X_test)
y_prob = lr.predict_proba(X_test)[:, 1]

print(confusion_matrix(y_test, y_hat))
print(accuracy_score(y_test, y_hat))
print(precision_score(y_test, y_hat))
print(recall_score(y_test, y_hat))
print(f1_score(y_test, y_hat))
print(roc_auc_score(y_test, y_prob))

In [ ]:
# 임계값 조정
for th in [0.3, 0.33, 0.36, 0.39, 0.42, 0.45, 0.48, 0.5]:
    y_hat = Binarizer(threshold=th).fit_transform(y_prob.reshape(-1, 1))
    print(confusion_matrix(y_test, y_hat))
    print(accuracy_score(y_test, y_hat))
    print(precision_score(y_test, y_hat))
    print(recall_score(y_test, y_hat))
    print(f1_score(y_test, y_hat))
    print(roc_auc_score(y_test, y_prob))